In this assignement, feel free to use the `sparse` module from `scipy`.

Use the cell below for your imports.

In [12]:
import numpy as np
from scipy.sparse import coo_matrix
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve

implement the function `mat_mul_coo` that takes two sparse matrices in `coo` and returns their product.

In [6]:
def mat_mul_coo(A, B):
    """
    Compute the product of two sparse matrices in COO format.

    Parameters
    ----------
    A : coo_matrix
        First sparse matrix in COO format.
    B : coo_matrix
        Second sparse matrix in COO format.

    Returns
    -------
    C : coo_matrix
        Product of A and B in COO format.
    """
    if A.shape[1] != B.shape[0]:
        raise ValueError("Matrices cannot be multiplied")

    A_data = A.data
    A_row = A.row
    A_col = A.col
    B_data = B.data
    B_row = B.row
    B_col = B.col

    # Create a dictionary to store the values of the product matrix
    C_dict = {}

    # Iterate over the non-zero entries of A
    for i in range(len(A_data)):
        # Iterate over the non-zero entries of B with the same row index as the current entry in A
        for j in range(len(B_data)):
            if A_col[i] == B_row[j]:
                # Multiply the corresponding values and add them to the dictionary
                key = (A_row[i], B_col[j])
                value = A_data[i] * B_data[j]
                if key in C_dict:
                    C_dict[key] += value
                else:
                    C_dict[key] = value

    # Create the COO matrix from the dictionary values
    C_data = np.array(list(C_dict.values()))
    C_row, C_col = zip(*C_dict.keys())
    C = coo_matrix((C_data, (C_row, C_col)), shape=(A.shape[0], B.shape[1]))

    return C

# Create two sparse matrices in COO format
A = coo_matrix([[1, 0, 0], [0, 2, 0], [0, 0, 3]])
B = coo_matrix([[4, 0], [0, 5], [0, 0]])

# Compute their product
C = mat_mul_coo(A, B)

# Print the resulting matrix in COO format
print(C.toarray())


[[ 4  0]
 [ 0 10]
 [ 0  0]]


implement the function `mat_mul_csr` that takes two sparse matrices in `csr` format and returns their product.

In [11]:
import numpy as np
from scipy.sparse import csr_matrix

def mat_mul_csr(A, B):
    """
    Compute the product of two sparse matrices in CSR format.

    Parameters
    ----------
    A : csr_matrix
        First sparse matrix in CSR format.
    B : csr_matrix
        Second sparse matrix in CSR format.

    Returns
    -------
    C : csr_matrix
        Product of A and B in CSR format.
    """
    if A.shape[1] != B.shape[0]:
        raise ValueError("Matrices cannot be multiplied")

    # Get the CSR data arrays for A and B
    A_data = A.data
    A_indices = A.indices
    A_indptr = A.indptr
    B_data = B.data
    B_indices = B.indices
    B_indptr = B.indptr

    # Create arrays to store the data, indices, and indptr values for the product matrix C
    C_data = []
    C_indices = []
    C_indptr = [0]

    # Iterate over the rows of A
    for i in range(A.shape[0]):
        # Create a dictionary to store the non-zero entries of the current row of C
        C_dict = {}

        # Iterate over the non-zero entries of the i-th row of A
        for k in range(A_indptr[i], A_indptr[i + 1]):
            # Iterate over the non-zero entries of the k-th column of B
            for j in range(B_indptr[A_indices[k]], B_indptr[A_indices[k] + 1]):
                if B_indices[j] in C_dict:
                    C_dict[B_indices[j]] += A_data[k] * B_data[j]
                else:
                    C_dict[B_indices[j]] = A_data[k] * B_data[j]

        # Add the non-zero entries of the current row of C to the data and indices arrays
        C_data.extend(C_dict.values())
        C_indices.extend(C_dict.keys())
        C_indptr.append(len(C_indices))

    # Create the CSR matrix from the data, indices, and indptr arrays
    C = csr_matrix((C_data, C_indices, C_indptr), shape=(A.shape[0], B.shape[1]))

    return C

# Create two sparse matrices in CSR format
A = csr_matrix([[1, 0, 0], [0, 2, 0], [0, 0, 3]])
B = csr_matrix([[4, 0], [0, 5], [0, 0]])

# Compute their product
C = mat_mul_csr(A, B)

# Print the resulting matrix in CSR format
print(C.toarray())


[[ 4  0]
 [ 0 10]
 [ 0  0]]


implement a function `solve_lin_sys` that takes a matrix `A` in `csr` format and a vector `b` as a numpy array and solves the system `Ax = b`.

In [15]:
def solve_lin_sys(A, b):
    """
    Solve the linear system Ax = b, where A is a sparse matrix in CSR format and
    b is a numpy array.

    Parameters
    ----------
    A : csr_matrix
        Sparse matrix in CSR format.
    b : ndarray
        Numpy array representing the right-hand side of the linear system.

    Returns
    -------
    x : ndarray
        Numpy array representing the solution of the linear system.
    """
    x = spsolve(A, b)
    return x